In [ ]:
import imp
compomics_import = imp.load_source('compomics_import', '../../compomics_import.py')
from IPython.core.display import HTML
css_file = '../../my.css'
HTML(open(css_file, "r").read())

# Classification

In this practical session we will build a classification model for gene splice site prediction. It is a problem arising in computational gene finding and concerns the recognition of splice sites that mark the boundaries between exons and introns in eukaryotes. Introns are spliced from premature mRNAs after transcription. The vast majority of splice sites are characterized by the presence of specific dimers on the intronic side of the splice site: GT for donor and AG for acceptor sites. Yet, only about 0.1-1% of all GT and AG occurrences in the genome represent true splice sites. 

Load the acceptor site data set in the file "C_elegans_acc_seq-1.csv".

Investigate the first 5 rows in the DataFrame.

There are only two columns. The column "sequence" contains a DNA sequence with lenght 22. The nucleotides at positions 11 and 12 in the sequence are always "A" and "G" respectively, so these positions are candidate gene acceptor sites. The column "target" indicates the class: 1 for "is acceptor site" and -1 for "is not acceptor site". The goal is to predict the target from the local context sequence of the candidate acceptor site. Let's see how many data points belong to each class:

In [ ]:
data['label'].value_counts()

First we need to convert the DNA sequence into features. To do this efficiently we use the function `apply()` of the pandas dataframe. With this function we can apply any other function to the rows or columns of the dataframe. Consider the following example dataframe:

In [ ]:
import numpy as np

example = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'))
example

Let's say we want to compute the difference between the maximum and minimum value in each row. We create a function that will have a row in a dataframe as input. The function returns the difference between the maximum and the minimum value in the row: 

In [ ]:
def function_to_apply(x):
    return np.max(x) - np.min(x)

Now we apply this function to the rows in the example dataframe:

In [ ]:
example.apply(function_to_apply,axis=1)

Find out how to compute difference between the maximum and minimum value in each column.

What does the following function do?

In [ ]:
def convert_sequence(x):
    return list(x['sequence'])

In the following code we create a new dataframe called `data_feaures` by applying the function `convert_sequence()` to the data set.

In [ ]:
cols = ["P"+str(i) for i in range(22)]
data_features = pd.DataFrame(list(data.apply(convert_sequence,axis=1)),columns=cols,dtype='category')

Investigate the first 5 rows.

Clearly we can't train a logistic regression model on categorical features. We need to transform these to numerically valued features. The following code transform each nucleotide in an integer.

In [ ]:
def convert_to_int(x):
    return x.cat.codes

data_features_int = data_features.apply(convert_to_int)

data_features_int.head(5)

Evaluate a logisitc regression model with hyperparameters $C=0.1$ on the data set `data_features_int` using 10-fold cross-validation. Use the `cross_val_score()` function to compute the mean accuracy of the CV-scores. 

Is this good generalization performance? To investigate this further let's split the data set into a train (2/3) and a test (1/3) by creating a list `folds` that will allow us to partition the rows in the data set into three parts.

Train a logistic regression model on the first two parts.

To predict the class labels for the third part (the test set) using this model we can use the following code.

In [ ]:
predictions = model.predict(data_features_int[folds==2])

Plot a histogram of the predictions.

Scikit-learn offers many metrics to evaluate models. These functions are contained in the `metrics` module of `sklearn`. Can you find how to compute the accuracy of these predictions?

An accuracy of around 90% seems like a good score. But is it? Let's consider a model that predicts class "-1" for all test points.

In [ ]:
predictions_zero = [-1]*len(data.label[folds==2])

What is the accuracy of these predictions?

So this should be a good score as well, even though we did not learn anything.

For classification tasks where the classes are highly imbalanced accuracy is not a good metric to evaluate the generalization performance. In fact, if there are 0.1% AG dinucleotides in a genome that are true acceptor sites then a model that predicts class "-1" for each AG would have an accuracy of 99.9%.

We have seen how a ROC curve plots the true positives rate against the false positives rate. Both these metrics focus on the positive class, in our case the true acceptor sites. These metrics are much more suitable to evalute the performance of models on tasks with highly imbalanced classes. To transform a ROC curve into one metric we can use the area under the curve (AUC). 

What is the AUC score of the predictions computed by the linear regression model we fitted?

To compute the AUC we need the predictions to be scores rather than class labels. For logistic regression these scores are the class probabilities predicted by the model. We can obtain these using the `predict_proba()` function of the `LogisticRegression` module.

In [ ]:
predictions = model.predict_proba(data_features_int[folds==2])

What does variable `predictions` contain?

The first and second column contains the predicted probabilities for class '-1' and '1' respectively. To compute the AUC we need to use the positive class probabilities. What is the AUC now?

Is this good generalization performance?

Transforming categorical features into ordered integers is maybe not a good idea as the nucleotides don't have any ordering. It is better to transform a categorical feature into one binary feature for each category. We can do this with the following code:

In [ ]:
def transform_features(data,column):
    tmp = pd.get_dummies(data[column])
    tmp.columns = [column+ '_' + x for x in tmp]
    return tmp

In [ ]:
to_concat = []
for c in data_features.columns.values:
    to_concat.append(transform_features(data_features,c))
data_features_bin = pd.concat(to_concat,axis=1)

Now we created a data set "data_features_bin" that contains features indicating the precense of a specific nucleotide at a specific position in the local context sequences. Investigate the first 5 rows.

We will use the same partitioning (into three parts) as we did for the data set `data_features_int`. Train a logistic regression model (again $C=0.1$) on the first two parts of the data set `data_features_bin` and evaluate the model on the third part of the data set using both accuracy and AUC.

This shows much better generalization performance. Use the `GridSearchCV()` function in scikit-learn to find a better value for hyperparameter `C` using cross-validation on the train set (first two parts).

`GridSearchCV` has an attribute called `best_estimator_` (go and look it up!). Use this attribute to evaluate the performance of the best model found by `GridSearchCV` on the third part. Compute both accuracy and AUC.